In [37]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
!pip install tensorflow-datasets
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalMaxPooling1D, TextVectorization, Embedding, Conv1D

print(tf.__version__)

2.20.0-dev20250418


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-04-25 18:03:02--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.06s   

2025-04-25 18:03:03 (6.02 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-04-25 18:03:03--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

In [41]:
train_file = pd.read_csv(train_file_path, sep="\t", names=["type", "message"])
train_file["type"] = train_file["type"].astype("category").cat.codes

test_file = pd.read_csv(test_file_path, sep="\t", names=["type", "message"])
test_file["type"] = test_file["type"].astype("category").cat.codes

train_file, test_file

(      type                                            message
 0        0  ahhhh...just woken up!had a bad dream about u ...
 1        0                           you can never do nothing
 2        0  now u sound like manky scouse boy steve,like! ...
 3        0  mum say we wan to go then go... then she can s...
 4        0  never y lei... i v lazy... got wat? dat day ü ...
 ...    ...                                                ...
 4174     0  just woke up. yeesh its late. but i didn't fal...
 4175     0  what do u reckon as need 2 arrange transport i...
 4176     1  free entry into our £250 weekly competition ju...
 4177     1  -pls stop bootydelious (32/f) is inviting you ...
 4178     0  tell my  bad character which u dnt lik in me. ...
 
 [4179 rows x 2 columns],
       type                                            message
 0        0  i am in hospital da. . i will return home in e...
 1        0         not much, just some textin'. how bout you?
 2        0  i probably won

In [7]:
MAX_VOCAB = 1000
MAX_LEN = 100

In [10]:
vec = TextVectorization(max_tokens=MAX_VOCAB, output_sequence_length=MAX_LEN, output_mode="int")
vec.adapt(train_file["message"].values)

In [38]:
model = Sequential([
    vec,
    Embedding(MAX_VOCAB, 64),
    Conv1D(64, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation="relu"),
    Dense(1, activation="sigmoid"),
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(train_file["message"].values, train_file["type"].values, validation_data=(test_file["message"].values, test_file["type"].values), epochs=10, batch_size=32)

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8663 - loss: 0.4476 - val_accuracy: 0.9749 - val_loss: 0.0863
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9818 - loss: 0.0600 - val_accuracy: 0.9820 - val_loss: 0.0627
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9927 - loss: 0.0269 - val_accuracy: 0.9828 - val_loss: 0.0543
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9965 - loss: 0.0152 - val_accuracy: 0.9820 - val_loss: 0.0575
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9992 - loss: 0.0065 - val_accuracy: 0.9820 - val_loss: 0.0574
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9995 - loss: 0.0033 - val_accuracy: 0.9828 - val_loss: 0.0595
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9996 - loss: 0.0017 - val_accuracy: 0.9828 - val_loss: 0.0621
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9999 - loss: 0.0013 - val_accuracy: 0.

In [39]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prob = model.predict(tf.constant([pred_text]))[0][0]
  return [float(prob), "spam" if prob > 0.4 else "ham"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
[3.156054049213708e-07, 'ham']


In [40]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
You passed the challenge. Great job!
